In [ ]:
import importlib
import utils 
import json
import geojson

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pylab as plt
import seaborn as sns; sns.set()

from shapely.geometry import Point

%matplotlib inline

In [ ]:
importlib.reload(utils)

df = utils.PrepareData().df
df.head()

In [ ]:
df.info()

In [ ]:
gdf = gpd.read_file("data/boundaries.geojson", driver='GeoJSON')
gdf.head()

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 15))
gdf.plot(facecolor="none",
         edgecolor="b",
         ax=ax)
         
f.suptitle('Seattle borders')
plt.savefig('figs/seattle_borders.png')
plt.show()

In [ ]:
df.head()

In [ ]:
df[['long', 'lat']].describe()

In [ ]:
points = [Point(df.iloc[i][['long', 'lat']]) for i in range(df.shape[0])]

In [ ]:
gdf['count'] = gdf.geometry.apply(lambda row: sum([row.contains(p) for p in points] ) )

In [ ]:
gdf.head()

In [ ]:
[gdf[gdf['OBJECTID'] == i] for i in range(gdf.shape[0])]

In [ ]:
g = json.loads(gdf.to_json())
n = len(g['features'])
max_coords = np.array([np.array(g['features'][i]['geometry']['coordinates'][0]).max(axis=0) for i in range(n)]).max(axis=0)
min_coords = np.array([np.array(g['features'][i]['geometry']['coordinates'][0]).min(axis=0) for i in range(n)]).min(axis=0)

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 15))
gdf.plot(column="count",
         edgecolor="k",
         colormap='jet',
         ax=ax,
         legend=True
         )

f.suptitle('Number of sales by districts')
plt.savefig('figs/number_of_sales_by_district.png')
plt.show()

In [ ]:
gdf1 = gdf.copy()
gdf1['sale_numbers'] = gdf1['SHAPE_Area'] / gdf1['count']
f, ax = plt.subplots(1, figsize=(15, 15))
gdf1.plot(column="sale_numbers",
         edgecolor="k",
         colormap='jet',
         ax=ax,
         legend=True
         )

f.suptitle('Number of sales per ')
# plt.savefig('figs/number_of_sales_by_district.png')
plt.show()

In [ ]:
df.head()

In [ ]:
ndf = df.copy()
def get_city_id(row):
    point = Point(row[['long', 'lat']])
    for i in range(gdf.shape[0]):
        if gdf.iloc[i].geometry.contains(point):
            return i
    else:
        return -1

ndf['city_id'] = ndf.apply(get_city_id, axis=1)

In [ ]:
ndf.head()

In [ ]:
ndf['city_id'].value_counts()

In [ ]:

ndf['price_per_sqft'] = ndf['price'] / ndf['sqft_living']
ndf.head()
gdf['price_per_sqft'] = ndf.groupby('city_id')['price_per_sqft'].mean()

In [ ]:
gdf.head()

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 15))
gdf.plot(column="price_per_sqft",
         edgecolor="k",
         colormap='jet',
         ax=ax,
         legend=True
         )
f.suptitle('Average price per square foot by districts')
plt.savefig('figs/average_price_per_sqft_by_district.png')
plt.show()